In [2]:
# !pip install boto3

# Initiation

In [3]:
import boto3
import json
import os
import sys
import time
sys.path.append("../")

from dotenv import load_dotenv

from utils.openaiClient import OpenAIClient

# Load the .env file (assuming it’s in the same folder)
load_dotenv()
AWS_REGION = os.getenv("AWS_REGION")

boto3.setup_default_session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=AWS_REGION
)

openai_client = OpenAIClient(api_key=os.getenv("OPENAI_API_KEY"))

# Extract from document

In [1]:
from textractClient import (
    start_job,
    is_job_complete, 
    get_job_results,
    save_results_to_file,
    load_textract_pages,
    extract_tables_from_blocks_unmerged,
    extract_forms_kv,
    write_tables_csv,
    write_forms_csv,
    build_llm_jsonl
)

s3_bucket = 'catalogs-for-extraction'
document_name = 'SampleCylindersCatalog'
document = f'{document_name}.pdf'
s3_path = f's3://{s3_bucket}/{document}'

job_id = "9c2c2e2e5a7c1c28c1f09b85da90c8413d7f9ecd4a8d37bc7ee9315a5cc357b7"
# job_id = start_job(s3_bucket, document, region=AWS_REGION)

In [ ]:
status = is_job_complete(job_id, region=AWS_REGION)
results_pages = get_job_results(job_id, region=AWS_REGION)
save_results_to_file(results_pages, f'textract_output_{document_name}.json')

Job status: SUCCEEDED


# Parse textract output

In [ ]:
TEXTRACT_JSON_PATH = f"textract_output_{document_name}.json"  # your saved file (list of responses with Blocks)
# --- Run the pipeline ---
pages = load_textract_pages(TEXTRACT_JSON_PATH)

# Flatten all Blocks across responses (Textract paginates API, not your PDF pages)
all_blocks = []
for part in pages:
    all_blocks.extend(part.get("Blocks", []))

output_dir = "output"
tables = extract_tables_from_blocks_unmerged(all_blocks, replicate_data=True, header_scan_rows=5)
forms = extract_forms_kv(all_blocks)

write_tables_csv(tables, output_dir)
write_forms_csv(forms, output_dir)
build_llm_jsonl(tables, output_dir)

len(tables), len(forms)


(11, 97)

In [27]:
import pandas as pd
pd.read_csv('tables_out/tables_page_3_table_1.csv')
# pd.read_csv('tables_consolidated.csv')
# pd.read_csv('forms_kv_pairs.csv')

,Material Grade/ Cylinder,Pressure Rating,Internal Volume,P,Ordering,col_6,"Dimensions, in. (mm)",col_8,Weight
0,Specification,psig (bar),cm³ ± 5 %,in.,Number,A,B,T,lb (kg)
1,NaN,NaN,NaN,NaN,Single-Ended,NaN,NaN,NaN,NaN
2,NaN,NaN,150,NaN,304L-05SF4-150,NaN,4.88 (124),NaN,1.1 (0.50)
3,304L SS/ DOT-4B 500,500 (34.4),300,1/4,304L-05SF4-300,2.00 (50.8),8.62 (219),0.093 (2.4),1.8 (0.82)
4,NaN,NaN,500,NaN,304L-05SF4-500,NaN,13.6 (345),NaN,2.7 (1.2)
5,NaN,NaN,NaN,NaN,Double-Ended,NaN,NaN,NaN,NaN
6,NaN,NaN,40,1/8,304L-HDF2-40,1.25 (31.8),3.88 (98.6),0.070 (1.8),0.31 (0.14)
7,NaN,NaN,50,NaN,304L-HDF4-50,NaN,3.75 (95.2),NaN,0.38 (0.17)
8,NaN,NaN,75,NaN,304L-HDF4-75,1.50 (38.1),4.94 (125),NaN,0.62 (0.28)
9,304L SS/ DOT-3E 1800,1800 (124),150,1/4,304L-HDF4-150,NaN,5.25 (133),0.093,0.94 (0.43)


# Using ChatGPT to convert all outputs into table

In [9]:
# Load and prepare data for OpenAI processing
import pandas as pd

# Load the tables data
tables_data = []
with open('output/tables_for_llm.jsonl', 'r') as f:
    for line in f:
        tables_data.append(json.loads(line.strip()))

# Load the forms data
forms_df = pd.read_csv('output/forms_kv_pairs.csv')

# Convert forms data to a more readable format
forms_text = "Key-Value Pairs from OCR:\n"
for _, row in forms_df.iterrows():
    forms_text += f"Page {row['page']}: {row['key']} = {row['value']}\n"

# Convert tables data to readable format
tables_text = "Table Data from OCR:\n"
for table in tables_data:
    tables_text += f"\nPage {table['page']}, Table {table['table_index']}:\n"
    tables_text += f"Headers: {', '.join(table['headers'])}\n"
    tables_text += "Rows:\n"
    for row in table['rows']:
        tables_text += f"  {row}\n"

print("Data loaded successfully!")
print(f"Tables: {len(tables_data)} entries")
print(f"Forms: {len(forms_df)} key-value pairs")


Data loaded successfully!
Tables: 6 entries
Forms: 97 key-value pairs


In [26]:
# Updated prompt that accounts for OCR data quality issues
prompt = f"""You are a helpful assistant that will help me organize product information from a catalog into a structured table format.

IMPORTANT: The data provided below comes from an OCR (Optical Character Recognition) service, which means:
- You must include all the existing products in the catalog, this is very important and can save lives.
- Text may contain recognition errors, typos, or formatting issues
- Numbers and units might be misread or inconsistently formatted
- Table structures may not be perfectly aligned
- Some information might be missing or fragmented
- You may need to make reasonable inferences and assumptions to fill gaps

I will provide you with two data sources:
1. Table data extracted from the catalog pages
2. Key-value pairs of additional product information

Please analyze these data sources and create a unified product table with the following columns:
- ordering_number: The product ordering/model number
- product_description: Brief description of what the product is
- material: Material grade/specification of the product
- dimensions: Key dimensions (A, B, T etc.) in inches (mm)
- weight: Weight in lb (kg)
- specs: Additional specifications as a JSON object

Please structure the data in a clear, consistent format. If any field is not available for a product, use null/empty values.

The output should be ONLY a JSON format with each product as an object containing the above fields.

{tables_data}

{forms_df}

Please process all the data and return a well-structured JSON array of product entries."""


In [45]:
web_version_messages = [
    {
        "role": "system", 
        "content": """You are an expert data-extraction and normalization assistant. You will receive information extracted from an engineering product catalog using OCR (AWS Textract). Your goal is to reconstruct a clean, consistent product table.

You will receive two data sources:
1. Tables data - Each line is a JSON object with:
   - page: page number
   - table_index: table number on page  
   - headers: list of column headers
   - rows: list of cell value lists

2. Forms key-value pairs - Additional fields found on pages (e.g. material, temperature)

You must:
1. Reconstruct products by:
   - Iterating over all tables
   - Treating each row as a product/model
   - Identifying ordering/part/catalog numbers as unique IDs

2. Assemble product specs by combining:
   - Non-empty columns as key-value pairs
   - Relevant form fields from same page
   - Normalized header names (lowercase snake_case)

3. Handle complex headers by:
   - Logically joining multi-row headers
   - Using section field for grouping headers

4. Output a JSON array with objects containing:
   - ordering_number: string
   - section: string (optional) 
   - page: number
   - table_index: number
   - specs: object of attributes
   - description_en: 1-2 sentence summary

Guidelines:
- Merge units (e.g. "4.88 (124)" -> "4.88 in (124 mm)")
- Keep values as strings
- Include all relevant specs
- Skip rows without ordering numbers
- Preserve hierarchical context
- Output only valid JSON"""
    },
    {
        "role": "user",
        "content": f"Please analyze this catalog data and create a unified product table following the above format:\n\nTables Data:\n{tables_data}\n\nForms Data:\n{forms_df}"
    }
]

In [27]:
# Create message object for OpenAI client call
messages = [
    {
        "role": "user",
        "content": prompt
    }
]

print(f"Message object created successfully! {len(prompt)} characters")


Message object created successfully! 11622 characters


In [39]:
find_ordering_number = [
    {
        "role": "user",
        "content": f"Find all the ordering numbers in the catalog, return a json array of the ordering numbers /n {tables_data} /n {forms_df}"
    }
]

In [56]:
# Call OpenAI client with JSON response format
print("Calling OpenAI client for JSON response...")
response = openai_client.chat_completion(
    messages=web_version_messages, 
    # messages=find_ordering_number, 
    model="gpt-4"
)

result = json.loads(response.choices[0].message.content)
print(f"Number of products extracted: {len(result) if isinstance(result, list) else 'Unknown'}")

# Save the result to a file
with open('unified_product_table_structured.json', 'w') as f:
    json.dump(result, f, indent=2)

print("Results saved to 'unified_product_table_structured.json'")

# Display first product as preview
if isinstance(result, list) and len(result) > 0:
    print("\nPreview of first product:")
    print(json.dumps(result[0], indent=2))


Calling OpenAI client for JSON response...


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [59]:
# result['products']
print(response.choices[0].message.content)

Here's the JSON array that was created from the given catalog data:

[{
	"ordering_number": "304L-05SF4-150",
	"section": "End / Inlet / 1/4 in. / male NPT",
	"page": 4,
	"table_index": 5,
	"specs": {
		"material_grade_cylinder_specification": "1/4 in. male NPT",
		"pressure_rating_psig_bar": "1/4 in. female NPT",
		"internal_volume_cm³_±_5_%": "SS-16DKM4-F4-A-1",
		"p_in": "0.218 (5.5)",
		"cylinder_ordering_number": "304L-05SF4-150",
		"dimensions_in_mm_b": "1.88 (47.7)",
		"a_bar_1_88_47_7_2_19_55_6": "1/4 in. male NPT",
		"weight_lb_kg": "With 1900 psig (130 bar) Rupture Disc"
	},
	"description_en": "Body made of seamless tubing provides consistent, superior, and long cycle life. Outage tubes help accommodate liquid thermal expansion in cylinders. Transportable Pressure Equipment Directive (TPED) compliant cylinder models available."
},
{
	"ordering_number": "SS-RTM4-F4-1",
	"section": "End / Inlet / 1/4 in. male NPT / 1/2 in. male NPT",
	"page": 4,
	"table_index": 6,
	"specs": {
	

In [ ]:
# Read the unified product table JSON file
df = pd.read_json('unified_product_table.json')
# df.to_csv('unified_product_table.csv', index=False)
specs_df = pd.json_normalize(df.specs)
# pd.concat([df, specs_df], axis=1)